<a href="https://colab.research.google.com/github/Hepman-git/Git_workshop/blob/main/Landlab_Tilted_grid_faulted_GoogleAI_Studio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # Import the necessary Landlab components
from landlab import RasterModelGrid
import numpy as np

# -----------------------
# 1. Model Parameters
# -----------------------
grid_size = 50  # Cell size in meters
model_length = 100000  # Model length in meters
model_width = 100000  # Model width in meters
tilt_angle = 4  # degrees
azimuth = 135  # degrees from North (Southeast)
fault_azimuth = 45  # degrees from North
fault_slip_rate = 0.001  # meters/year (1 mm/year)
fault_uplift_rate = 0.0001  # meters/year (0.1 mm/year)
shallower_depth = 300 # meters bellow sea level

# -----------------------
# 2. Create the Grid
# -----------------------
num_rows = int(model_width / grid_size)
num_cols = int(model_length / grid_size)
grid = RasterModelGrid((num_rows, num_cols), xy_spacing=grid_size)

print(f"Number of rows: {grid.number_of_node_rows}")
print(f"Number of columns: {grid.number_of_node_columns}")
print(f"Number of nodes: {grid.number_of_nodes}")
print(f"Cell size: {grid.dx} meters")

# -----------------------
# 3. Define Initial Topography (Tilted Plane)
# -----------------------
tilt_angle_rad = np.radians(tilt_angle)
azimuth_rad = np.radians(azimuth)
x_slope = np.tan(tilt_angle_rad) * np.sin(azimuth_rad)
y_slope = np.tan(tilt_angle_rad) * np.cos(azimuth_rad)

x = grid.x_of_node
y = grid.y_of_node
elevation = -(x * x_slope + y * y_slope) + shallower_depth #Tilted plane + shallower depth

grid.at_node['topographic__elevation'] = elevation

print(
    "The minimum depth is "
    + str(round(grid.at_node["topographic__elevation"].min(), 2))
    + " m, and the maximum is "
    + str(round(grid.at_node["topographic__elevation"].max(), 2))
    + " m."
)

# -----------------------
# 4. Implement Fault Offset
# -----------------------
fault_azimuth_rad = np.radians(fault_azimuth)
x_center = model_length / 2
y_center = model_width / 2

distance_to_fault = (x - x_center) * np.cos(fault_azimuth_rad) + (y - y_center) * np.sin(fault_azimuth_rad)

horizontal_displacement_rate = np.where(distance_to_fault > 0, fault_slip_rate, -fault_slip_rate)
uplift_rate = np.where(distance_to_fault > 0, fault_uplift_rate, 0)

grid.at_node['horizontal_displacement_rate'] = horizontal_displacement_rate
grid.at_node['uplift_rate'] = uplift_rate

print(
    "The minimum horizontal displacement rate is "
    + str(round(grid.at_node["horizontal_displacement_rate"].min(), 4))
    + " m/yr, and the maximum is "
    + str(round(grid.at_node["horizontal_displacement_rate"].max(), 4))
    + " m/yr."
)

print(
    "The minimum uplift rate is "
    + str(round(grid.at_node["uplift_rate"].min(), 5))
    + " m/yr, and the maximum is "
    + str(round(grid.at_node["uplift_rate"].max(), 5))
    + " m/yr."
)

# -----------------------
# 5. Set Boundary Conditions
# -----------------------
grid.set_fixed_gradient_boundaries_at_grid_edges(
     True, # north
     False, # east
    False, # south
    False, # west
)

# Set the fixed gradient at the north
grid.at_boundary['topographic__elevation'][grid.boundary_nodes[0]] = elevation[grid.boundary_nodes[0]]